In [6]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

In [7]:
df1 = pd.read_csv('data/features_basicas.csv', low_memory=False, index_col='person')
#df1["proprosion"] = np.where(df1["cant_viewed_product"]>0, (df1["cant_checkouts"]/df1["cant_viewed_product"]),0)


In [8]:
df2 = pd.read_csv('data/vistas_promedio.csv', low_memory=False, index_col='person')
#df3 = pd.read_csv('data/featureUsuarioRealizaAlMenos10EventosEn20Minutos.csv', low_memory=False, index_col='person')
df4 = pd.read_csv('data/features_mas_vistos.csv', low_memory=False, index_col='person')
df5 = pd.read_csv('data/features_compras.csv', low_memory=False, index_col='person')
df6 = pd.read_csv('data/features_checkout.csv', low_memory=False, index_col='person')
df7 = pd.read_csv('data/features_time.csv', low_memory=False, index_col='person')
df8 = pd.read_csv('data/features_time_2.csv', low_memory=False, index_col='person')
df9 = pd.read_csv('data/features_time_3.csv', low_memory=False, index_col='person')
#df10 = pd.read_csv('data/paises_features.csv', low_memory=False, index_col='person')
df11 = pd.read_csv('data/city_features.csv', low_memory=False, index_col='person')

df_unidos = df1.join(df2).fillna(0)
#df_unidos = df_unidos.join(df3).fillna(0)
df_unidos = df_unidos.join(df4).fillna(0)
df_unidos = df_unidos.join(df5).fillna(0)
df_unidos = df_unidos.join(df6).fillna(0)
df_unidos = df_unidos.join(df7).fillna(0)
df_unidos = df_unidos.join(df8).fillna(0)
df_unidos = df_unidos.join(df9).fillna(0)
#df_unidos = df_unidos.join(df10).fillna(0)
df_unidos = df_unidos.join(df11).fillna(0)
df_unidos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38829 entries, 4886f805 to 80aea0a0
Data columns (total 23 columns):
cant_conversions               38829 non-null float64
cant_checkouts                 38829 non-null float64
cant_viewed_product            38829 non-null float64
cant_searched_product          38829 non-null float64
cant_ad_campaign_hit           38829 non-null float64
cant_brand_listing             38829 non-null float64
total_sesiones                 38829 non-null int64
promedio_eventos_por_sesion    38829 non-null float64
mas_prom                       38829 non-null float64
vio_mas_apple                  38829 non-null float64
vio_mas_samsung                38829 non-null float64
vio_mas_motorola               38829 non-null float64
compro_apple                   38829 non-null float64
check_apple                    38829 non-null float64
check_samsung                  38829 non-null float64
timestamp                      38829 non-null int64
prom_timestamp            

In [9]:
labels = pd.read_csv('data/labels_training_set.csv', low_memory=False, index_col='person')

df_test = df_unidos.join(labels, how= "inner")
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19414 entries, ad93850f to d8d6b0da
Data columns (total 24 columns):
cant_conversions               19414 non-null float64
cant_checkouts                 19414 non-null float64
cant_viewed_product            19414 non-null float64
cant_searched_product          19414 non-null float64
cant_ad_campaign_hit           19414 non-null float64
cant_brand_listing             19414 non-null float64
total_sesiones                 19414 non-null int64
promedio_eventos_por_sesion    19414 non-null float64
mas_prom                       19414 non-null float64
vio_mas_apple                  19414 non-null float64
vio_mas_samsung                19414 non-null float64
vio_mas_motorola               19414 non-null float64
compro_apple                   19414 non-null float64
check_apple                    19414 non-null float64
check_samsung                  19414 non-null float64
timestamp                      19414 non-null int64
prom_timestamp            

In [10]:
features = df_test.columns.tolist()
features.remove('label')
print(features)

['cant_conversions', 'cant_checkouts', 'cant_viewed_product', 'cant_searched_product', 'cant_ad_campaign_hit', 'cant_brand_listing', 'total_sesiones', 'promedio_eventos_por_sesion', 'mas_prom', 'vio_mas_apple', 'vio_mas_samsung', 'vio_mas_motorola', 'compro_apple', 'check_apple', 'check_samsung', 'timestamp', 'prom_timestamp', 'sum_timestamp', 'city_Belo Horizonte', 'city_Bras\xc3\xadlia', 'city_Rio de Janeiro', 'city_Salvador', 'city_S\xc3\xa3o Paulo']


In [11]:
labels['label'].value_counts(normalize=True)

0    0.949521
1    0.050479
Name: label, dtype: float64

In [12]:
rf = RandomForestClassifier(n_estimators=10000, n_jobs=-1, min_samples_split=150,\
                            class_weight='balanced')

#  Partimos los datos que tenemos para entrenar en dos partes, una para entrenar el modelo (80% de los datos)
#  y la otra parte se usará para probar el modelo (el 20% restante)
X_train, X_test, Y_train, Y_test = train_test_split(df_test[features],\
                                                    df_test['label'], test_size=0.20)

In [13]:
rf.fit(X_train,Y_train)
rf.score(X_test,Y_test)

0.80324491372650009

In [14]:
Y_pred = rf.predict(X_test)
print(roc_auc_score(Y_test, Y_pred))

0.767664479155


In [15]:
#  Esto nos tira data sobre la precisión, cuántas de las cosas que clasificamos salieron bien clasificadas, y f1

Y_truths = Y_test
print(classification_report(Y_truths, Y_pred))

             precision    recall  f1-score   support

          0       0.98      0.81      0.89      3670
          1       0.18      0.73      0.29       213

avg / total       0.94      0.80      0.85      3883



In [16]:
print(confusion_matrix(Y_truths, Y_pred))

[[2964  706]
 [  58  155]]


In [17]:
feature_importances = pd.DataFrame(rf.feature_importances_,\
                                   index = X_train.columns,\
                                    columns=['importance']).sort_values('importance',ascending=False)

In [18]:
feature_importances.head(50)

,importance
cant_checkouts,0.288996
prom_timestamp,0.126756
timestamp,0.124524
sum_timestamp,0.069746
cant_viewed_product,0.054056
check_apple,0.043060
total_sesiones,0.039980
cant_brand_listing,0.039617
check_samsung,0.038834
promedio_eventos_por_sesion,0.037781


## Summit

In [19]:
df_submit = pd.read_csv('data/trocafone_kaggle_test.csv', low_memory=False, index_col='person')
df_events = df_submit.join(df_unidos, how='inner')

In [20]:
df_events

,cant_conversions,cant_checkouts,cant_viewed_product,cant_searched_product,cant_ad_campaign_hit,cant_brand_listing,total_sesiones,promedio_eventos_por_sesion,mas_prom,vio_mas_apple,...,check_apple,check_samsung,timestamp,prom_timestamp,sum_timestamp,city_Belo Horizonte,city_Brasília,city_Rio de Janeiro,city_Salvador,city_São Paulo
person,,,,,,,,,,,,,,,,,,,,,
4886f805,0.0,1.0,4.0,1.0,0.0,0.0,1,9.000000,0.0,0.0,...,0.0,1.0,1526603430000000000,1526602376249999872,1.221282e+19,0.0,0.0,0.0,0.0,0.0
0297fc1e,0.0,7.0,404.0,6.0,29.0,4.0,89,6.370787,1.0,1.0,...,1.0,0.0,1527508212000000000,1522498682203065856,7.947443e+20,0.0,0.0,1.0,0.0,0.0
2d681dd8,0.0,1.0,13.0,1.0,1.0,5.0,2,13.000000,0.0,0.0,...,0.0,0.0,1527450169000000000,1527141699500000256,3.359712e+19,0.0,0.0,0.0,0.0,0.0
cccea85e,0.0,1.0,739.0,1.0,15.0,7.0,17,49.176471,1.0,0.0,...,0.0,0.0,1527777510000000000,1526707378753567488,1.177091e+21,0.0,0.0,0.0,0.0,0.0
4c8a8b93,0.0,2.0,177.0,9.0,14.0,8.0,14,18.357143,1.0,1.0,...,1.0,0.0,1526949198000000000,1526693704236841472,3.480862e+20,0.0,0.0,0.0,0.0,0.0
29ebb414,0.0,1.0,8.0,11.0,4.0,0.0,4,8.750000,0.0,1.0,...,1.0,0.0,1526837857000000000,1526651492240000256,3.816629e+19,0.0,0.0,1.0,0.0,0.0
3dc1950f,0.0,3.0,595.0,2.0,3.0,6.0,27,24.888889,1.0,0.0,...,0.0,0.0,1527614864000000000,1525503572702921472,9.397102e+20,0.0,0.0,0.0,0.0,0.0
8ea4c165,0.0,3.0,97.0,17.0,13.0,5.0,7,22.714286,1.0,0.0,...,0.0,1.0,1526739719000000000,1526196972719424768,2.121414e+20,0.0,0.0,0.0,0.0,1.0
d8cfe234,1.0,3.0,39.0,8.0,0.0,4.0,5,12.400000,1.0,0.0,...,0.0,1.0,1527017412000000000,1526337924946428672,8.547492e+19,0.0,0.0,0.0,0.0,0.0


In [21]:
kaggle_pred = rf.predict_proba(df_events)
kaggle_pred
proba_de_comprar = [x[1] for x in kaggle_pred]
series = pd.Series(proba_de_comprar)
df_submit['label'] = series.values

In [22]:
df_submit.to_csv('submit.csv')


In [23]:
df_submit.label.mean()

0.26833172202988104

In [73]:
0.34460859613130107

0.34460859613130107